In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, mean, stddev

In [16]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("Projeto 1 BigData") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [17]:
raw_df = spark.read.csv("./dados_caged_2022.csv", header=True, sep=",")

In [18]:
raw_df.show(5)

+----+---+---+------------+------------+----------------+------------------+--------+---------+-----------------+-----+-----------------+--------+----+---------------+--------------------+-----------------+----------------+-------------------------------+--------------------------+--------------+-------------------------------+------------------+-----------------+--------------------+
| ano|mes| uf|id_municipio|cnae_2_secao|cnae_2_subclasse|saldo_movimentacao|cbo_2002|categoria|grau_de_instrucao|idade|horas_contratuais|raca_cor|sexo|tipo_empregador|tipo_estabelecimento|tipo_movimentacao|tipo_deficiencia|indicador_trabalho_intermitente|indicador_trabalho_parcial|salario_mensal|tamanho_estabelecimento_janeiro|indicador_aprendiz|origem_informacao|indicador_fora_prazo|
+----+---+---+------------+------------+----------------+------------------+--------+---------+-----------------+-----+-----------------+--------+----+---------------+--------------------+-----------------+----------------+-

# Parsing Data

In [19]:
# Definindo os mapas de parsing
sexo_parse_map = {'1': 'HOMEM', '3': 'MULHER', '9': 'NAO_IDENTIFICADO'}
tipo_empregador_parse_map = {'0': 'CNPJ', '2': 'CPF', '9': 'NAO_IDENTIFICADO'}
saldo_mov_parse_map = {'1': 'ADMITIDO', '-1': 'DEMITIDO'}

# Função para aplicar parsing nos DataFrames
def parse_dataframe(df):
    df = df.drop('id_municipio', 'cbo_2002', 'categoria', 'raca_cor', 'tipo_movimentacao', 
                 'tipo_deficiencia', 'indicador_trabalho_intermitente', 'indicador_trabalho_parcial', 
                 'tamanho_estabelecimento_janeiro', 'indicador_aprendiz', 'origem_informacao', 
                 'indicador_fora_prazo', 'tipo_estabelecimento')

    df = df.withColumn('sexo', when(col('sexo') == '1', 'HOMEM')
                                .when(col('sexo') == '3', 'MULHER')
                                .when(col('sexo') == '9', 'NAO_IDENTIFICADO'))
    
    df = df.withColumn('tipo_empregador', when(col('tipo_empregador') == '0', 'CNPJ')
                                         .when(col('tipo_empregador') == '2', 'CPF')
                                         .when(col('tipo_empregador') == '9', 'NAO_IDENTIFICADO'))

    df = df.withColumn('saldo_movimentacao', when(col('saldo_movimentacao') == '1', 'ADMITIDO')
                                             .when(col('saldo_movimentacao') == '-1', 'DEMITIDO'))

    return df

# Definindo o arquivo de entrada e saída
output_filename = './dados_caged_2022_parsed.csv'

# Aplicando a função de parsing
parsed_df = parse_dataframe(raw_df)

In [20]:
parsed_df.show(5)

+----+---+---+------------+----------------+------------------+-----------------+-----+-----------------+-----+---------------+--------------+
| ano|mes| uf|cnae_2_secao|cnae_2_subclasse|saldo_movimentacao|grau_de_instrucao|idade|horas_contratuais| sexo|tipo_empregador|salario_mensal|
+----+---+---+------------+----------------+------------------+-----------------+-----+-----------------+-----+---------------+--------------+
|2023|  4| SP|           H|         4930202|          DEMITIDO|                6|   37|             44.0|HOMEM|           CNPJ|       2060.89|
|2023|  4| SP|           H|         4930202|          DEMITIDO|                7|   28|             44.0|HOMEM|           CNPJ|       2428.16|
|2023|  4| SP|           H|         4930202|          DEMITIDO|                7|   24|             44.0|HOMEM|           CNPJ|        1898.0|
|2023|  4| SP|           H|         4930202|          DEMITIDO|                7|   27|             44.0|HOMEM|           CNPJ|       2361.87|

# Removing Outliers

In [23]:
# Função para calcular Z-score e remover outliers
def remove_outliers(df, column):
    # Calculando a média e desvio padrão
    stats = df.select(mean(col(column)).alias('mean'), stddev(col(column)).alias('stddev')).collect()[0]
    mean_value = stats['mean']
    stddev_value = stats['stddev']

    # Adicionando a coluna z_score
    df = df.withColumn('z_' + column, (col(column) - mean_value) / stddev_value)

    # Filtrando os outliers
    df = df.filter((col('z_' + column) <= 3) | col(column).isNull())

    return df

parsed_df = remove_outliers(parsed_df, 'salario_mensal')

In [25]:
parsed_df.drop('z_salario_mensal')

# Salvando o DataFrame transformado em um novo CSV
parsed_df.write.csv(output_filename, header=True, mode='overwrite')

In [68]:
cb_df = spark.read.csv('./cesta_basica_2022.csv', header=True, sep=',') 

cb_df.show(5)

+-----+-------+------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----+
|Sigla| Código|   Município|2022.01|2022.02|2022.03|2022.04|2022.05|2022.06|2022.07|2022.08|2022.09|2022.10|2022.11|2022.12|_c15|
+-----+-------+------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+----+
|   AC|1200013|  Acrelândia|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|NULL|
|   AC|1200054|Assis Brasil|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|NULL|
|   AC|1200104|   Brasiléia|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|NULL|
|   AC|1200138|      Bujari|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|NULL|
|   AC|1200179|    Capixaba|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NUL

24/05/25 23:57:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Sigla, Código, Município, 2022.01, 2022.02, 2022.03, 2022.04, 2022.05, 2022.06, 2022.07, 2022.08, 2022.09, 2022.10, 2022.11, 2022.12, 
 Schema: Sigla, Código, Município, 2022.01, 2022.02, 2022.03, 2022.04, 2022.05, 2022.06, 2022.07, 2022.08, 2022.09, 2022.10, 2022.11, 2022.12, _c15
Expected: _c15 but found: 
CSV file: file:///home/hellhat/Documents/facul/BigData/cesta_basica_2022.csv


In [69]:
cb_df = cb_df.drop('_c15')
cb_df = cb_df.filter(cb_df["`2022.01`"].isNotNull())

In [70]:
monthly_columns = [f"`2022.{str(i).zfill(2)}`" for i in range(1, 13)]
monthly_columns_no_parsing = [f"2022.{str(i).zfill(2)}" for i in range(1, 13)]


# Calculate the mean of the monthly columns
cb_df = cb_df.withColumn("media_cesta_basica", sum(col(c) for c in monthly_columns) / len(monthly_columns))

# Drop the original monthly columns
cb_df = cb_df.drop(*monthly_columns_no_parsing)

In [74]:
cb_df = cb_df.drop('Código', 'Município')
cb_df.show(30)
cb_df.write.csv('parsed_cesta_basica.csv', header=True, mode='overwrite')

+-----+------------------+
|Sigla|media_cesta_basica|
+-----+------------------+
|   BA| 566.3316666666667|
|   CE|             631.8|
|   DF| 699.4575000000001|
|   ES| 701.6608333333334|
|   GO|          668.6175|
|   MG|          661.0075|
|   MS| 713.1366666666667|
|   PA| 613.7741666666667|
|   PB|            563.41|
|   PE| 576.2008333333334|
|   PR| 691.9408333333332|
|   RJ|             730.0|
|   RN| 579.3108333333332|
|   RS|          747.3175|
|   SC| 751.2283333333334|
|   SE| 529.0558333333335|
|   SP| 762.2308333333331|
+-----+------------------+



24/05/26 00:02:04 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/05/26 00:02:15 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/05/26 00:02:18 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/05/26 00:02:19 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not immediately visible in the filesystem.
24/05/26 00:02:19 WARN BasicWriteTaskStatsTracker: Expected 1 files, but only saw 0. This could be due to the output format not writing empty files, or files being not imme